**Mounting Google Drive and changing directory**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/til-ai-camp/baseline_code'

In [ ]:
os.chdir('/content/drive/My Drive/W.A.S.D Warriors/')

## Download and Explore the Dataset

In [ ]:
# Creating two directories - "data" and "data/trainset_11classes_0_00000" 
#!mkdir data && mkdir data/trainset_11classes_0_00000
# Downloading the ai-camp competition dataset
#!wget -N https://ai-camp.s3-us-west-2.amazonaws.com/trainset_11classes_0_00000.zip
# Unzip the data into the folder "data/trainset_11classes_0_00000"
#!unzip -qq -n trainset_11classes_0_00000.zip -d data/trainset_11classes_0_00000
# Switch directory to "data/trainset_11classes_0_00000" and show its content
!cd data/trainset_11classes_0_00000 && ls

train  train_new  train_new_combined  val


In [ ]:
import os

base_dir = 'data/trainset_11classes_0_00000'

# Directory to our training data
train_folder = os.path.join(base_dir, 'train_new_combined')

# Directory to our validation data
val_folder = os.path.join(base_dir, 'val')

Now, let's find out the total number of images we have in each `train`, `val` and `test`.

In [ ]:
# List folders and number of files
print("Directory, Number of files")
for root, subdirs, files in os.walk('data/trainset_11classes_0_00000/train_new_combined'):
    print(root, len(files))

Directory, Number of files
data/trainset_11classes_0_00000/train_new_combined 0
data/trainset_11classes_0_00000/train_new_combined/Spiderman 101
data/trainset_11classes_0_00000/train_new_combined/EaglePose 96
data/trainset_11classes_0_00000/train_new_combined/HighKneel 96
data/trainset_11classes_0_00000/train_new_combined/ChestBump 56
data/trainset_11classes_0_00000/train_new_combined/HandShake 93
data/trainset_11classes_0_00000/train_new_combined/HulkSmash 99
data/trainset_11classes_0_00000/train_new_combined/HandGun 104
data/trainset_11classes_0_00000/train_new_combined/Dabbing 101
data/trainset_11classes_0_00000/train_new_combined/KungfuCrane 100
data/trainset_11classes_0_00000/train_new_combined/KoreanHeart 97
data/trainset_11classes_0_00000/train_new_combined/Salute 100
data/trainset_11classes_0_00000/train_new_combined/WarriorPose 104
data/trainset_11classes_0_00000/train_new_combined/KungfuSalute 102
data/trainset_11classes_0_00000/train_new_combined/ChairPose 101
data/trainset_

## Data Preprocessing

Let's set up data generators that will read images from our source folders and convert them to float32 tensors. We'll have one generator for each training and validation folders.

### Batch
Our generators will yield batches of `32` images of size `299 x 299` and their labels.

### Feature scaling
Recall that in our MNIST/CIFAR-10 exercises, data that goes into a neural network should be normalised in a way that is easier to be processed by the network. In our case, we will preprocess our images by normalising the pixels values to be in the 0 to 1 range. This happens by dividing each pixel value by 255 and this process is known as data normalisation or rescaling.

### Generator - ImageDataGenerator
To rescale the data, we use `keras.preprocessing.image.ImageDataGenerator` class with the `rescale` parameter. This class will also allow us to instantiate generators of augmented image batches (and their labels) via `.flow_from_directory(directory)`. These generators can then be used with the Keras model methods that accept data generators as inputs such as `fit_generator`, `evaluate_generator` and `predict_generator`. We used data augmentation for the training image generator. To find out more about how to do image augmentation in keras, go [here](https://keras.io/preprocessing/image/).

**Defining Random Eraser**

In [ ]:
import numpy as np


def get_random_eraser(p=0.5, s_l=0.02, s_h=0.1, r_1=0.1, r_2=1/0.1, v_l=0, v_h=1, pixel_level=True):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Batch size
bs = 32

# All images will be resized to this value
image_size = (299, 299)

# All images will be rescaled by 1./255. We apply data augmentation here.
train_datagen = ImageDataGenerator(rescale=1./255,
                                   brightness_range= [0.5,1.5],
                                   horizontal_flip=True,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest',
                                   preprocessing_function=get_random_eraser(v_l=0, v_h=1),)

val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
print("Preparing generator for train dataset")
train_generator = train_datagen.flow_from_directory(
    directory= train_folder, # This is the source directory for training images 
    target_size=image_size, # All images will be resized to value set in image_size
    batch_size=bs,
    class_mode='categorical')

# Flow validation images in batches of 32 using val_datagen generator
print("Preparing generator for validation dataset")
val_generator = val_datagen.flow_from_directory(
    directory= val_folder, 
    target_size=image_size,
    batch_size=bs,
    class_mode='categorical')

Using TensorFlow backend.


Preparing generator for train dataset
Found 1441 images belonging to 15 classes.
Preparing generator for validation dataset
Found 347 images belonging to 15 classes.


**Feature Extraction**

1) Import Pre-trained models without dense layers

2) Freeze All Layers

3) Added in Dense, Dropout with regularizers

In [ ]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, MaxPooling2D
from keras import backend as K
from keras import regularizers

Xcep = Xception(weights='imagenet', include_top=False)

x = Xcep.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.1)(x)
predictions = Dense(15, activation='softmax')(x)

model = Model(inputs = Xcep.input, outputs=predictions)

for layer in Xcep.layers:
  layer.trainable = False
  
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Setting Up Checkpoints

Let's setup a [checkpoint](https://keras.io/callbacks/) to help us monitor the validation accuracy as the model trains. This checkpoint will save the model with best validation accuracy seen so far.

In [ ]:
from keras.callbacks import ModelCheckpoint

bestValidationCheckpointer = ModelCheckpoint('Xcep15.hdf5', monitor='val_acc', save_best_only=True, verbose=1)

Train till it stagnates (~50epochs)

In [ ]:
history = model.fit_generator(
        train_generator, # train generator has 973 train images
        steps_per_epoch=train_generator.samples // bs + 1,
        epochs=100,
        validation_data=val_generator, # validation generator has 253 validation images
        validation_steps=val_generator.samples // bs + 1,
        callbacks=[bestValidationCheckpointer]
)

Epoch 1/100
46/46 [==============================] - 651s 14s/step - loss: 8.5619 - acc: 0.0945 - val_loss: 4.3728 - val_acc: 0.1556

Epoch 00001: val_acc improved from -inf to 0.15562, saving model to Xcep15.hdf5
Epoch 2/100
46/46 [==============================] - 50s 1s/step - loss: 3.6129 - acc: 0.1516 - val_loss: 3.0535 - val_acc: 0.1037

Epoch 00002: val_acc did not improve from 0.15562
Epoch 3/100
46/46 [==============================] - 63s 1s/step - loss: 2.9305 - acc: 0.1923 - val_loss: 2.6704 - val_acc: 0.1988

Epoch 00003: val_acc improved from 0.15562 to 0.19885, saving model to Xcep15.hdf5
Epoch 4/100
46/46 [==============================] - 63s 1s/step - loss: 2.9798 - acc: 0.1767 - val_loss: 2.7391 - val_acc: 0.1787

Epoch 00004: val_acc did not improve from 0.19885
Epoch 5/100
46/46 [==============================] - 63s 1s/step - loss: 2.7847 - acc: 0.2134 - val_loss: 2.7000 - val_acc: 0.1816

Epoch 00005: val_acc did not improve from 0.19885
Epoch 6/100
46/46 [======

OSError: ignored

In [ ]:
for i, layer in enumerate(model.layers):
  print(i, layer.name)

0 input_2
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv2_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d_5
13 block2_pool
14 batch_normalization_5
15 add_13
16 block3_sepconv1_act
17 block3_sepconv1
18 block3_sepconv1_bn
19 block3_sepconv2_act
20 block3_sepconv2
21 block3_sepconv2_bn
22 conv2d_6
23 block3_pool
24 batch_normalization_6
25 add_14
26 block4_sepconv1_act
27 block4_sepconv1
28 block4_sepconv1_bn
29 block4_sepconv2_act
30 block4_sepconv2
31 block4_sepconv2_bn
32 conv2d_7
33 block4_pool
34 batch_normalization_7
35 add_15
36 block5_sepconv1_act
37 block5_sepconv1
38 block5_sepconv1_bn
39 block5_sepconv2_act
40 block5_sepconv2
41 block5_sepconv2_bn
42 block5_sepconv3_act
43 block5_sepconv3
44 block5_sepconv3_bn
45 add_16
46 block6_sepconv1_act
47 block6_sepconv1
48 block6_sepconv1_bn
49 block6_sepconv2_act
50 block6_sepconv2
51 block6_sepconv2_bn
52 block6_se

**Unfreezing Layers for Fine-Tuning** 

Choose to unfreeze about 2/3 of the layers

In [ ]:
for layer in model.layers[:56]:
  layer.trainable = False
for layer in model.layers[56:]:
  layer.trainable = True

Let's summarise the model architecture:

In [ ]:
import pandas as pd

layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])  

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at...,input_2,False
1,<keras.layers.convolutional.Conv2D object at 0...,block1_conv1,False
2,<keras.layers.normalization.BatchNormalization...,block1_conv1_bn,False
3,<keras.layers.core.Activation object at 0x7fad...,block1_conv1_act,False
4,<keras.layers.convolutional.Conv2D object at 0...,block1_conv2,False
5,<keras.layers.normalization.BatchNormalization...,block1_conv2_bn,False
6,<keras.layers.core.Activation object at 0x7fad...,block1_conv2_act,False
7,<keras.layers.convolutional.SeparableConv2D ob...,block2_sepconv1,False
8,<keras.layers.normalization.BatchNormalization...,block2_sepconv1_bn,False
9,<keras.layers.core.Activation object at 0x7fad...,block2_sepconv2_act,False


**Change Optimizer**

In [ ]:
from keras.optimizers import SGD

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.001, momentum=0.9, decay=1e-11),
              metrics=['accuracy'])

## Model Training 


In [ ]:
history = model.fit_generator(
        train_generator, # train generator has 973 train images
        steps_per_epoch=train_generator.samples // bs + 1,
        epochs=200,
        validation_data=val_generator, # validation generator has 253 validation images
        validation_steps=val_generator.samples // bs + 1,
        callbacks=[bestValidationCheckpointer]
)

Epoch 1/200
46/46 [==============================] - 74s 2s/step - loss: 0.3989 - acc: 0.9182 - val_loss: 1.3363 - val_acc: 0.6830

Epoch 00001: val_acc did not improve from 0.79539
Epoch 2/200
46/46 [==============================] - 69s 2s/step - loss: 0.5011 - acc: 0.8747 - val_loss: 1.2785 - val_acc: 0.6945

Epoch 00002: val_acc did not improve from 0.79539
Epoch 3/200
46/46 [==============================] - 69s 1s/step - loss: 0.5252 - acc: 0.8842 - val_loss: 1.8054 - val_acc: 0.5937

Epoch 00003: val_acc did not improve from 0.79539
Epoch 4/200
36/46 [======================>.......] - ETA: 13s - loss: 0.5184 - acc: 0.8819

KeyboardInterrupt: ignored

**Plotting Graph**

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()